In [1]:
import json
import scipy
import numpy as np
import scipy.sparse
import tensorflow as tf
import tensorflow.keras.backend as K

from tensorflow.keras import Sequential, layers
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.utils import class_weight
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# Limit GPU memory usage
for device in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(device, True)

In [2]:
tf.__version__

'2.4.0'

## Load/import packages

In [4]:
# load Aff-Wild2 Features
X_train = scipy.sparse.load_npz("data/features/train_features_RGB_AW2.npz") #CSR Matrix
X_val = scipy.sparse.load_npz("data/features/val_features_RGB_AW2.npz") #CSR Matrix
y_train = np.load("data/labels/train_labels_RGB_AW2.npy") #Numpy array
y_val = np.load("data/labels/val_labels_RGB_AW2.npy") #Numpy array

In [5]:
# Load AFEW7.0 features
X_test_AF7 = scipy.sparse.load_npz("data/features/features_RGB_AF7.npz") #CSR Matrix
y_test_AF7 = np.load("data/labels/labels_RGB_AF7.npy") #Numpy array

In [6]:
# Split the validation set into validation and test
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size = 0.5, random_state = 1234, shuffle=True)

# Prepare data

In [7]:
train_label_ints = np.argmax(y_train, axis=1) 
class_weights = class_weight.compute_class_weight(
        "balanced", np.unique(train_label_ints), train_label_ints.flatten()
    )
class_weights = {i : class_weights[i] for i in range(len(class_weights))}

C:\Users\ion\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1 2 3 4 5 6], y=[0 0 0 ... 6 6 6] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [10]:
with open("weights.json", 'w') as fp:
    json.dump(class_weights, fp)

In [32]:
# Set early stopping
es = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.0025,
    patience=6,
    verbose=1,
    mode="auto",
    restore_best_weights=True,
)

# Define and train model

In [33]:
model = Sequential()
model.add(layers.InputLayer(input_shape=4608,))
model.add(layers.Dense(32))
model.add(layers.Dense(7, activation="softmax", name="Dense_Output"))
model.compile(optimizer="adam", loss=CategoricalCrossentropy(), metrics=["accuracy"])

model.summary()
# Fit model to training set and evaluate
batchsize = 512
history = model.fit(
    train_features_AW2,
    train_labels_AW2,
    batch_size=batchsize,
    class_weight=class_weights,
    validation_data=(val_features_AW2, val_labels_AW2),
    epochs=5,
    verbose=1,
    shuffle=True,
)


# Save model
tf.keras.Model.save(
    model,
    filepath=f"data/models/models_with_extractedfeatures_vgg19block5/FCN.h5",
)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 32)                147488    
_________________________________________________________________
Dense_Output (Dense)         (None, 7)                 231       
Total params: 147,719
Trainable params: 147,719
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
1801/1801 [==============================] - 17s 9ms/step - loss: 2.6186 - accuracy: 0.6183 - val_loss: 3.6355 - val_accuracy: 0.3608
Epoch 2/5
1801/1801 [==============================] - 16s 8ms/step - loss: 0.4474 - accuracy: 0.7599 - val_loss: 3.0565 - val_accuracy: 0.3730
Epoch 3/5
1801/1801 [==============================] - 15s 8ms/step - loss: 0.4136 - accuracy: 0.7683 - val_loss: 3.1195 - val_accuracy: 0.3904
Epoch 4/5
1801/1801 [==============================] - 14s 7ms/step - loss: 0.

## Evaluate on test sets

In [40]:
NN = tf.keras.models.load_model(
    filepath=f"data/models/models_with_extractedfeatures_vgg19block5/FCN.h5",
    compile=True,
)


# Check model summary
NN.summary()

# Evaluate on test set of AW2 to get test scores
csvlog_AW2_test = tf.keras.callbacks.CSVLogger(
    f"data/models/test_scores/FCN_AW2_test_scores.csv",
    separator=",",
    append=False,
)
NN.evaluate(
    X_test, y_test, batch_size=batchsize, callbacks=[csvlog_AW2_test],
)

# Get F1 scores for AW2 test set
test_pred = NN.predict(X_test, verbose=0)

# Convert one hot encoding to integers
test_pred = np.argmax(test_pred, axis=1)

# Reshape back to (frame, label)
test_true = np.argmax(y_test, axis=1)

f1scores_test = {
    avg: f1_score(test_pred, test_true, average=avg) for avg in [None, "macro"]
}
f1scores_test[None] = f1scores_test.get(None).tolist()
print(f1scores_test)

with open(
    f"data/models/test_scores/FCN_AW2_test_F1scores.json", "w",
) as fp:
    json.dump(f1scores_test, fp)

##################################################################################

# Evaluate on test set of AF7 to get test scores for cross-dataset performance
csvlog_AF7 = tf.keras.callbacks.CSVLogger(
    f"data/models/test_scores/FCN_AF7_test_scores.csv",
    separator=",",
    append=False,
)
NN.evaluate(
    X_test_AF7, y_test_AF7, batch_size=batchsize, callbacks=[csvlog_AF7],
)

# Get F1 scores for AF7 test set
test_pred = NN.predict(X_test_AF7, verbose=0)

# Convert one hot encoding to integers
test_pred = np.argmax(test_pred, axis=1)

# Reshape back to (frame, label)
test_true = np.argmax(y_test_AF7, axis=1)

f1scores_test = {
    avg: f1_score(test_pred, test_true, average=avg) for avg in [None, "macro"]
}
f1scores_test[None] = f1scores_test.get(None).tolist()
print(f1scores_test)

with open(
    f"data/models/test_scores/FCN_AF7_test_F1scores.json", "w",
) as fp:
    json.dump(f1scores_test, fp)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 32)                147488    
_________________________________________________________________
Dense_Output (Dense)         (None, 7)                 231       
Total params: 147,719
Trainable params: 147,719
Non-trainable params: 0
_________________________________________________________________
156/156 [==============================] - 0s 2ms/step - loss: 3.0780 - accuracy: 0.4067
{None: [0.5747922785350302, 0.04759898904802022, 0.010303967027305513, 0.05227369315767106, 0.34970800412229475, 0.1168451801363194, 0.2569468505301088], 'macro': 0.20120985179382142}
131/131 [==============================] - 0s 2ms/step - loss: 4.4993 - accuracy: 0.2175
{None: [0.31015295287761085, 0.12989771393438043, 0.049733231707317076, 0.05375914836992682, 0.31831638563521686, 0.17792163209258988, 0.08081945069